In [1]:
import sys
import sklearn
import statistics
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
print("Python: {}".format(sys.version))

Python: 3.7.3 (default, Apr 24 2019, 15:29:51) [MSC v.1915 64 bit (AMD64)]


In [2]:
train_set = pd.read_csv("tcd ml 2019-20 income prediction training (with labels).csv", index_col = 0)
test_set = pd.read_csv("tcd ml 2019-20 income prediction test (without labels).csv", index_col = 0)

In [3]:
train_set.head()

,Year of Record,Gender,Age,Country,Size of City,Profession,University Degree,Wears Glasses,Hair Color,Body Height [cm],Income in EUR
Instance,,,,,,,,,,,
1,1997.0,0,41.0,Belarus,1239930,steel workers,Bachelor,0,Blond,193,61031.94416
2,1996.0,other,41.0,Singapore,1603504,safe event coordinator,Master,0,Black,186,91001.32764
3,2018.0,other,28.0,Norway,1298017,receivables/payables analyst,PhD,1,Brown,170,157982.17670
4,2006.0,other,33.0,Cuba,751903,fleet assistant,No,1,Black,171,45993.75793
5,2010.0,female,46.0,United Arab Emirates,95389,lead trainer,0,0,Blond,188,38022.16217


### Description of Dataset

In [4]:
train_set.describe()

,Year of Record,Age,Size of City,Wears Glasses,Body Height [cm],Income in EUR
count,111552.000000,111499.000000,1.119930e+05,111993.000000,111993.000000,1.119930e+05
mean,1999.421274,37.345304,8.388538e+05,0.500531,175.220192,1.092138e+05
std,11.576382,16.036694,2.196879e+06,0.500002,19.913889,1.498024e+05
min,1980.000000,14.000000,7.700000e+01,0.000000,94.000000,-5.696906e+03
25%,1989.000000,24.000000,7.273400e+04,0.000000,160.000000,3.077169e+04
50%,1999.000000,35.000000,5.060920e+05,1.000000,174.000000,5.733917e+04
75%,2009.000000,48.000000,1.184501e+06,1.000000,190.000000,1.260936e+05
max,2019.000000,115.000000,4.999251e+07,1.000000,265.000000,5.285252e+06


In [5]:
# sns.set(style="ticks", color_codes=True)

# sns.catplot(x="University Degree", y="Income in EUR", data=train_set)
# sns.catplot(x="Profession", y="Income in EUR", data=train_set)
# sns.catplot(x="Gender", y="Income in EUR", data=train_set)
# sns.catplot(x="Country", y="Income in EUR", data=train_set)
# sns.catplot(x="Hair Color", y="Income in EUR", data=train_set)

## One Hot Encoding
 --for non numerical data

In [6]:
train_set["train"] = 1
test_set["train"] = 0
train_db = pd.concat([train_set,test_set])

C:\Users\vansh\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


# Data Cleaning
1. Removing NULL values
2. Noise
3. Inconsistent Data

In [7]:
train_db["Year of Record"] = train_db["Year of Record"].fillna(statistics.mode(train_db['Year of Record'])).astype('int')
train_db["Age"] = train_db["Age"].fillna(statistics.mode(train_db['Age'])).astype('int')

1. **Gender**

In [8]:
train_db["Gender"] = train_db["Gender"].fillna(statistics.mode(train_db.Gender))
dfDummies = pd.get_dummies(train_db["Gender"], prefix = 'gender')
dfDummies.sum()

gender_0           1179
gender_female     44913
gender_male       83262
gender_other      44674
gender_unknown    11195
dtype: int64

In [9]:
train_db = pd.concat([train_db, dfDummies], axis=1)
train_db = train_db.drop(columns=["Gender"])

2. **Country**

In [10]:
train_db["Country"] = train_db["Country"].fillna(statistics.mode(train_db.Country))
dfDummies = pd.get_dummies(train_db["Country"], prefix = 'country')
dfDummies.sum().tail()

country_Venezuela     159
country_Vietnam         1
country_Yemen         105
country_Zambia        790
country_Zimbabwe     1437
dtype: int64

In [11]:
train_db = pd.concat([train_db, dfDummies], axis=1)
train_db = train_db.drop(columns=["Country"])

3. **Profession**

In [12]:
train_db["Profession"] = train_db["Profession"].fillna(statistics.mode(train_db.Profession))
dfDummies = pd.get_dummies(train_db["Profession"], prefix = 'profession')
dfDummies.sum().tail()

profession_workforce planning intern         42
profession_writer                            50
profession_x-ray technician                  47
profession_yardmaster                        31
profession_youth initiatives lead advisor    46
dtype: int64

In [13]:
train_db = pd.concat([train_db, dfDummies], axis=1)
train_db = train_db.drop(columns=["Profession"])

4. **University Degree**

In [14]:
train_db["University Degree"] = train_db["University Degree"].fillna(statistics.mode(train_db['University Degree']))
dfDummies = pd.get_dummies(train_db["University Degree"], prefix = 'degree')
dfDummies.sum().tail()

degree_0            1156
degree_Bachelor    83313
degree_Master      44831
degree_No          44697
degree_PhD         11226
dtype: int64

In [15]:
train_db = pd.concat([train_db, dfDummies], axis=1)
train_db = train_db.drop(columns=["University Degree"])

5. **Hair Color**

In [16]:
train_db["Hair Color"] = train_db["Hair Color"].fillna(statistics.mode(train_db['Hair Color']))
dfDummies = pd.get_dummies(train_db["Hair Color"], prefix = 'haircolor')
dfDummies.sum().tail()

haircolor_Black      83205
haircolor_Blond      44645
haircolor_Brown      44920
haircolor_Red        11335
haircolor_Unknown     1073
dtype: int64

In [17]:
train_db = pd.concat([train_db, dfDummies], axis=1)
train_db = train_db.drop(columns=["Hair Color"])

In [18]:
train = train_db[train_db["train"] ==1].drop(columns=["Income in EUR","Income", "train"], axis =1)
test = train_db[train_db["train"] ==0].drop(columns=["Income in EUR","Income", "train"], axis =1)
train_income = train_db[train_db["train"] ==1]["Income in EUR"]

In [19]:
train["Year of Record"] = train["Year of Record"].fillna(statistics.mode(train['Year of Record']))
train["Age"] = train["Age"].fillna(statistics.mode(train['Age']))

# MODEL CREATION

In [28]:
submit = pd.read_csv("tcd ml 2019-20 income prediction submission file.csv", index_col= 0)

### Linear Regression

In [21]:
model = linear_model.LinearRegression()
lmodel = model.fit(train, train_income)

y_pred = lmodel.predict(test)

In [33]:
submit.to_csv(r"C:\Users\vansh\TCD MAI\Machine Learning\IndividualAssignment\MLbasic\prediction.csv", index = True)

### Random Forest

In [20]:
train.head()

,Age,Body Height [cm],Size of City,Wears Glasses,Year of Record,gender_0,gender_female,gender_male,gender_other,gender_unknown,...,degree_Bachelor,degree_Master,degree_No,degree_PhD,haircolor_0,haircolor_Black,haircolor_Blond,haircolor_Brown,haircolor_Red,haircolor_Unknown
Instance,,,,,,,,,,,,,,,,,,,,,
1,41,193,1239930,0,1997,1,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
2,41,186,1603504,0,1996,0,0,0,1,0,...,0,1,0,0,0,1,0,0,0,0
3,28,170,1298017,1,2018,0,0,0,1,0,...,0,0,0,1,0,0,0,1,0,0
4,33,171,751903,1,2006,0,0,0,1,0,...,0,0,1,0,0,1,0,0,0,0
5,46,188,95389,0,2010,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [31]:
model = RandomForestRegressor(random_state=0, n_estimators=50)
lmodel = model.fit(train, train_income)

y_pred = lmodel.predict(test)

In [32]:
submit["Income"] = y_pred
submit.head()

,Income
Instance,
111994,32819.461361
111995,21039.942027
111996,35247.108240
111997,117408.245807
111998,40895.330765


### Submission

In [29]:
submit["Income"] = y_pred
submit.head()

,Income
Instance,
111994,41101.295779
111995,20157.472989
111996,34349.225034
111997,119508.143724
111998,30172.357219


In [24]:
submit.head()

,Income
Instance,
111994,26513.228917
111995,-32380.357196
111996,45828.513053
111997,169534.173142
111998,5551.777255


## Split data into train test

In [26]:
train_set.columns

Index(['Year of Record', 'Gender', 'Age', 'Country', 'Size of City',
       'Profession', 'University Degree', 'Wears Glasses', 'Hair Color',
       'Body Height [cm]', 'Income in EUR', 'train'],
      dtype='object')

In [34]:
temp = pd.get_dummies(train_set, columns = ["Profession","University Degree","Gender","Hair Color", "Country"])

In [35]:
temp.head()

,Year of Record,Age,Size of City,Wears Glasses,Body Height [cm],Income in EUR,train,Profession_.net developer,Profession_.net software developer,Profession_Accountant,...,Country_United Arab Emirates,Country_United Kingdom,Country_Uruguay,Country_Uzbekistan,Country_Vanuatu,Country_Venezuela,Country_Vietnam,Country_Yemen,Country_Zambia,Country_Zimbabwe
Instance,,,,,,,,,,,,,,,,,,,,,
1,1997.0,41.0,1239930,0,193,61031.94416,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1996.0,41.0,1603504,0,186,91001.32764,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2018.0,28.0,1298017,1,170,157982.17670,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2006.0,33.0,751903,1,171,45993.75793,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2010.0,46.0,95389,0,188,38022.16217,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train, y_test = train_test_split(train_set, train_set["Income in EUR"], test_size=0.30, random_state=42)
model = LogisticRegression()
lomodel = model.fit(X_train, y_train)
y_pred = lomodel.predict(X_test)

C:\Users\vansh\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: could not convert string to float: 'Brown'

In [6]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred)

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import model_selection
X_train, X_test, y_train, y_test = train_test_split(train, train_income, test_size=0.33, random_state=42)

In [35]:
model_selection.KFold(n_splits=10, random_state=seed)

KFold(n_splits=10, random_state=7, shuffle=False)

In [43]:
models = []
# models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
# models.append(('LDA', LinearDiscriminantAnalysis()))
# models.append(('KNN', KNeighborsClassifier()))
# models.append(('CART', DecisionTreeClassifier()))
# models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))
# evaluate each model in turn
results = []
names = []
seed =7
scoring = 'accuracy'
kfold = model_selection.KFold(n_splits=10, random_state=seed)
for name, model in models:
    print(name)
    cv_results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

SVM


C:\Users\vansh\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:528: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


ValueError: Unknown label type: 'continuous'